In [1]:
import sys, os
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from scipy.stats import ttest_ind_from_stats
from scipy.spatial import distance
import random
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
np.random.seed(0)

In [2]:
K = [2, 4, 8, 16, 32]
data1 = pd.read_csv('digits-embedding.csv', header=None)

In [3]:
d2 = [2, 4, 6, 7]
d3 = [6, 7]
data2 = data1[data1[1].isin(d2)]
data3= data1[data1[1].isin(d3)]

In [4]:
def kmeans(data, K):
    N = len(data)
    label = [0 for i in range(len(data))]
    count = 0
    centroid_idx = np.random.randint(0, N, size=K)
    centroid = []
    for i in centroid_idx:
        centroid.append((data.iloc[i][2], data.iloc[i][3]))
    while True:
        update = 0
        x = [0.0 for i in range(K)]
        y = [0.0 for i in range(K)]
        label_count = [0 for i in range(K)]
        idx = 0
        for index, row in data.iterrows():
            # print index, row[2], row[3]
            coor = (row[2], row[3])
            candidate_label = []
            for c in centroid:
                candidate_label.append(distance.euclidean(coor, c))
            # Get the label of the current point
            l = candidate_label.index(min(candidate_label))
            x[l] += row[2]
            y[l] += row[3]
            label_count[l] += 1
            
            # Check if there's an update in each round
            if l != label[idx]:
                label[idx] = l
                update += 1
            
            idx += 1
                
        count += 1
        
        if count == 50 or update == 0:
            break
        
        # Update the centroid if they does not meet the stop criterion
        for i in range(len(centroid)):
            if label_count[i] > 0:
                centroid[i] = (1.0 * x[i]/label_count[i], 1.0 * y[i]/label_count[i])
        
        print count, update
    
    return label, centroid

In [5]:
def get_stat(data, centroid, label, K):
    N = len(data)
    WC_SSD = 0
    S = []
    d = data.iloc[:,2:4]
    data_matrix = np.array(d)
    count = 0
    for index, row in data.iterrows():
        # Update the WC_SSD
        WC_SSD += np.square(distance.euclidean((row[2], row[3]), centroid[label[count]]))
        # Update the SC
        idx = [i for i in range(N) if label[i] == label[count]]
        #print idx
        SC_A = np.sum(np.sqrt(np.sum(np.square(data_matrix[idx,:] - data_matrix[count,:]), axis = 1)))
        SC_B = np.sum(np.sqrt(np.sum(np.square(data_matrix - data_matrix[count,:]), axis = 1))) - SC_A
        SC_A /= len(idx) - 1
        SC_B /= N - len(idx)
        S.append((SC_B-SC_A)/max(SC_A, SC_B))
        count += 1
    SC = np.sum(S)/len(S)
    print 'WC_SSD:', '%.2f' % WC_SSD
    print 'SC:', '%.2f' % SC
    return WC_SSD, SC

In [6]:
# Initial the centroid
label_1 = {}
label_2 = {}
label_3 = {}

centroid_1 = {}
centroid_2 = {}
centroid_3 = {}

WC_SSD_1 = []
WC_SSD_2 = []
WC_SSD_3 = []

SC_1 = []
SC_2 = []
SC_3 = []

for k in K:
    print "K =", k
    # Dataset 1
    label_1[k], centroid_1[k] = kmeans(data1, k)
    WC_SSD, SC = get_stat(data1, centroid_1[k], label_1[k], k)
    WC_SSD_1.append(WC_SSD)
    SC_1.append(SC)
    # Dataset 2
    np.random.seed(0)
    label_2[k], centroid_2[k] = kmeans(data2, k)
    WC_SSD, SC = get_stat(data2, centroid_2[k], label_2[k], k)
    WC_SSD_2.append(WC_SSD)
    SC_2.append(SC)
    # Dataset 3
    label_3[k], centroid_3[k] = kmeans(data3, k)
    WC_SSD, SC = get_stat(data3, centroid_3[k], label_3[k], k)
    WC_SSD_3.append(WC_SSD)
    SC_3.append(SC)
    
print WC_SSD_1
print SC_1
print WC_SSD_2
print SC_2
print WC_SSD_3
print SC_3

K = 2
1 17750
2 3287
3 2163
4 1068
5 579
6 408
7 329
8 209
9 133
10 100
11 61
12 34
13 18
14 14
15 13
16 17
17 17
18 13
19 16
20 14
21 7
22 8
23 6
24 3
25 1
26 1
WC_SSD: 8983900.00
SC: 0.37
1 3929
2 65
WC_SSD: 4211155.69
SC: 0.49
1 2101
WC_SSD: 340372.42
SC: 0.82
K = 4
1 16184
2 3295
3 2524
4 1710
5 977
6 553
7 360
8 272
9 153
10 121
11 95
12 97
13 91
14 101
15 102
16 82
17 79
18 67
19 56
20 53
21 46
22 49
23 45
24 43
25 38
26 40
27 44
28 52
29 60
30 60
31 71
32 87
33 92
34 117
35 140
36 177
37 193
38 210
39 229
40 208
41 212
42 199
43 204
44 179
45 164
46 158
47 125
48 105
49 75
WC_SSD: 4319783.51
SC: 0.54
1 4258
2 739
3 442
4 822
5 1017
6 502
7 39
8 4
WC_SSD: 623865.31
SC: 0.78
1 3392
2 378
3 110
4 57
5 27
6 32
7 9
8 4
9 2
WC_SSD: 223070.04
SC: 0.84
K = 8
1 16785
2 3253
3 2106
4 1606
5 1371
6 1272
7 1039
8 742
9 423
10 301
11 233
12 190
13 169
14 148
15 127
16 105
17 99
18 77
19 43
20 32
21 31
22 32
23 25
24 23
25 14
26 12
27 13
28 10
29 11
30 4
31 4
32 6
33 10
34 4
35 4
36 2
37 3
38